In [1]:
from data_processing import *

Seed set to 42


making new app


In [4]:
filepath = 'lstm_on_bouy.pth'
input_size = 3 # number of features
hidden_size = 64 # number of features in hidden state
num_layers = 3 # number of stacked lstm layers
num_classes = 2
lstm = LSTM(num_classes, input_size, hidden_size, num_layers).to(device)
lstm.load_state_dict(torch.load(filepath, map_location=device))

<All keys matched successfully>

In [6]:
def Predict(pond_id, n_ahead):
    """
    修改后的预测函数，适用于每10分钟采样的24小时连续数据
    
    Args:
        pond_id: 池塘ID
        n_ahead: 预测未来多少个时间步（每步10分钟）
    
    Returns:
        dataX, do, future_predicts, train_size, val_size, mean, std
    """
    # 获取数据
    data = getData_5feature_tensor(pond_id)
    future_predicts = []
    _, _, _, _, _, _, dataX, dataY, train_size, val_size, mean, std = Load_Data(data)
    do = data.iloc[:, :]

    # 使用最后4个观测值作为初始序列
    lastDay = do.tail(4) 
    last_hour_minute = lastDay['hour_minute'].iloc[-1]
    hold = lastDay
    lastDay = norimalize(lastDay, mean, std)
    
    # 获取最后的日期时间
    last_date = data.index[-1]
    last_date = pd.to_datetime(data.index[-1], format='%Y%m%d_%H:%M:%S')
    
    predicted_datetimes = []
    cur_hour_minute = last_hour_minute

    # 递归预测n_ahead步
    for i in range(n_ahead):
        # 计算下一个时间点（每10分钟增加）
        next_time_minutes = (cur_hour_minute * 60 + 10) % (24 * 60)  # 转换为分钟，加10分钟，24小时循环
        next_hour_minute = next_time_minutes / 60.0  # 转换回小时格式
        
        # 更新日期时间（每10分钟）
        last_date = last_date + timedelta(minutes=10)
        predicted_datetimes.append(last_date)
        
        cur_hour_minute = next_hour_minute

        # 转换为张量进行预测
        last_sequence = convert_tensor(lastDay)
        
        # 模型预测
        lstm.eval()
        with torch.no_grad():
            future_predict = lstm(last_sequence)
        
        # 处理预测输出
        future_predict = future_predict[0, :, :]  # 假设输出形状为 (1, seq_len, features)
        future_predict = future_predict.detach().cpu().numpy()
        
        # 由于模型只输出2个特征（do, temp），但标准化时可能有3个特征
        # 需要为第三个特征（hour_minute）添加值
        if future_predict.shape[1] == 2:  # 如果模型只输出2个特征
            # 添加hour_minute特征
            hour_minute_column = np.full((future_predict.shape[0], 1), next_hour_minute)
            future_predict = np.concatenate((future_predict, hour_minute_column), axis=1)
        
        # 反标准化
        future_predict_de = denormalize(future_predict, mean, std)
        
        # 确保hour_minute特征正确
        future_predict_de[:, 2] = next_hour_minute  # 第三列是hour_minute
        
        future_predicts.append(future_predict_de)

        # 重新标准化用于下一次预测
        future_predict_de_norm = norimalize(future_predict_de, mean, std)
        
        # 更新序列：移除最旧的观测，添加新预测
        lastDay = np.append(lastDay, future_predict_de_norm, axis=0)
        lastDay = lastDay[1:, :]  # 移除第一行，保持序列长度为4

    future_predicts = np.array(future_predicts)
    
    return dataX, do, future_predicts, train_size, val_size, mean, std

In [7]:
dataX, do, future_predicts, train_size, val_size, mean, std = Predict(1,6)
future_predicts

array([[[16.58543243, 32.55634295, 18.31666667]],

       [[16.24583733, 32.44251152, 18.48333333]],

       [[16.02803298, 32.36708425, 18.65      ]],

       [[15.83529017, 32.29930953, 18.81666667]],

       [[15.69300579, 32.24686311, 18.98333333]],

       [[15.58182483, 32.20530929, 19.15      ]]])